In [1]:
import torch
import torch_geometric
import math
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F

In [2]:
# 读取训练数据和测试数据
dataTrain = []
dataTest = []

path = 'D:/dataAMLGraph'

# 指定日期
date = '2022-09-01'

# 遍历一天内的小时
for hour in range(24):
    file_name = f"subGraph_{date}_{hour:02d}0000.pt"  # 构造文件名
    file_path = f"{path}/{file_name}"
    
    try:
        subGraph = torch.load(file_path)
        if hour <= 17:   # 0-17 小时作为训练集
            dataTrain.append(subGraph)
        else:            # 18-23 小时作为测试集
            dataTest.append(subGraph)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

print(f"Training graphs: {len(dataTrain)}, Testing graphs: {len(dataTest)}")



Training graphs: 18, Testing graphs: 6


In [3]:
dataTrain[0]

Data(x=[2077023, 3], edge_index=[2, 1380940], edge_attr=[4, 1380940], y=[1380940])

In [4]:
dataTrain[0].edge_attr[3].max()

tensor(14.)

In [6]:
from models import GCNWithEdge

In [7]:
model = GCNWithEdge(
    inFeat=dataTrain[0].x.shape[1],
    hiddenFeat=32,
    outFeat=16,
    embeddingDims=[20, 20],
    fchidden=32,
    vdim=2,
    finalDim=2,
    dropout=0.5
)



In [ ]:
import torch
from typing import Tuple, Dict, Any, Optional, Union, List
import numpy as np
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def calculate_metrics(y_true: torch.Tensor, y_pred: torch.Tensor, average: str = 'binary') -> Dict[str, float]:
    """
    计算分类指标
    
    Args:
        y_true: 真实标签
        y_pred: 预测标签
        average: 多分类时的平均方式 ('binary', 'micro', 'macro', 'weighted')
    
    Returns:
        dict: 包含各种指标的字典
    """
    y_true_np = y_true.cpu().numpy()
    y_pred_np = y_pred.cpu().numpy()
    
    metrics = {}
    
    try:
        # 精确率
        metrics['precision'] = precision_score(y_true_np, y_pred_np, average=average, zero_division=0)
        # 召回率
        metrics['recall'] = recall_score(y_true_np, y_pred_np, average=average, zero_division=0)
        # F1分数
        metrics['f1'] = f1_score(y_true_np, y_pred_np, average=average, zero_division=0)
        # 准确率
        metrics['accuracy'] = (y_pred == y_true).float().mean().item()
        
        # 混淆矩阵（对于二分类）
        if len(torch.unique(y_true)) == 2:
            tn, fp, fn, tp = confusion_matrix(y_true_np, y_pred_np).ravel()
            metrics.update({
                'true_negative': tn,
                'false_positive': fp,
                'false_negative': fn,
                'true_positive': tp
            })
            
    except Exception as e:
        print(f"计算指标时出错: {e}")
        metrics.update({
            'precision': 0.0,
            'recall': 0.0,
            'f1': 0.0,
            'accuracy': 0.0
        })
    
    return metrics


def train_model(
    model: torch.nn.Module,
    data_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    device: torch.device
) -> Tuple[float, Dict[str, Any]]:
    """
    使用DataLoader训练模型
    """
    model.train()
    total_loss = 0.0
    all_predictions = []
    all_targets = []
    
    for data in data_loader:
        data = data.to(device)
        
        # 确保标签是long类型
        if data.y.dtype != torch.long:
            data.y = data.y.long()
        
        # 前向传播
        optimizer.zero_grad()
        output = model(data)
        
        # 计算损失
        loss = criterion(output, data.y)
        
        # 反向传播
        loss.backward()
        optimizer.step()
        
        # 统计信息
        total_loss += loss.item()
        
        # 收集预测和真实标签
        pred = output.argmax(dim=1)
        all_predictions.append(pred)
        all_targets.append(data.y)
    
    # 计算指标
    all_pred = torch.cat(all_predictions)
    all_target = torch.cat(all_targets)
    metrics = calculate_metrics(all_target, all_pred)
    metrics['loss'] = total_loss / len(data_loader)
    metrics['num_graphs'] = len(data_loader)
    metrics['total_samples'] = all_target.size(0)
    
    return metrics['loss'], metrics


def test_model(
    model: torch.nn.Module,
    data_loader: DataLoader,
    criterion: torch.nn.Module,
    device: torch.device,
    return_predictions: bool = False
) -> Tuple[float, Dict[str, Any]]:
    """
    使用DataLoader测试模型性能
    """
    model.eval()
    total_loss = 0.0
    all_predictions = []
    all_targets = []
    all_outputs = []
    
    with torch.no_grad():
        for data in data_loader:
            data = data.to(device)
            
            # 确保标签是long类型
            if data.y.dtype != torch.long:
                data.y = data.y.long()
            
            # 前向传播
            output = model(data)
            
            # 计算损失
            loss = criterion(output, data.y)
            total_loss += loss.item()
            
            # 收集预测和真实标签
            pred = output.argmax(dim=1)
            all_predictions.append(pred)
            all_targets.append(data.y)
            all_outputs.append(output)
    
    # 计算指标
    all_pred = torch.cat(all_predictions)
    all_target = torch.cat(all_targets)
    metrics = calculate_metrics(all_target, all_pred)
    metrics['loss'] = total_loss / len(data_loader)
    metrics['num_graphs'] = len(data_loader)
    metrics['total_samples'] = all_target.size(0)
    
    if return_predictions:
        metrics['predictions'] = all_pred.cpu()
        metrics['targets'] = all_target.cpu()
        metrics['outputs'] = torch.cat(all_outputs).cpu()
    
    return metrics['loss'], metrics


def print_detailed_metrics(metrics: Dict[str, Any], phase: str = "Test"):
    """
    打印详细的评估指标
    """
    print(f"\n{phase} 详细指标:")
    print(f"损失: {metrics.get('loss', 0):.4f}")
    print(f"准确率: {metrics.get('accuracy', 0):.4f}")
    print(f"精确率: {metrics.get('precision', 0):.4f}")
    print(f"召回率: {metrics.get('recall', 0):.4f}")
    print(f"F1分数: {metrics.get('f1', 0):.4f}")
    
    # 如果是二分类，打印混淆矩阵
    if 'true_positive' in metrics:
        print(f"\n混淆矩阵:")
        print(f"真阳性(TP): {metrics['true_positive']}")
        print(f"假阳性(FP): {metrics['false_positive']}")
        print(f"假阴性(FN): {metrics['false_negative']}")
        print(f"真阴性(TN): {metrics['true_negative']}")
        
        # 计算额外指标
        if metrics['true_positive'] + metrics['false_positive'] > 0:
            precision = metrics['true_positive'] / (metrics['true_positive'] + metrics['false_positive'])
            print(f"精确率(手动计算): {precision:.4f}")
        
        if metrics['true_positive'] + metrics['false_negative'] > 0:
            recall = metrics['true_positive'] / (metrics['true_positive'] + metrics['false_negative'])
            print(f"召回率(手动计算): {recall:.4f}")


In [27]:
from torch.utils.data import Dataset
from torch_geometric.loader import DataLoader
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score

# 1. 包装 dataset
class MyGraphDataset(Dataset):
    def __init__(self, data_list):
        self.data_list = data_list

    def __len__(self):
        return len(self.data_list)

    def __getitem__(self, idx):
        return self.data_list[idx]

# 2. 分析类别分布
def analyze_class_distribution(data_list, name="Dataset"):
    all_labels = torch.cat([data.y for data in data_list])
    class_counts = torch.bincount(all_labels)
    total = len(all_labels)
    print(f"\n{name} 类别分布:")
    for i, count in enumerate(class_counts):
        ratio = count / total
        print(f"类别 {i}: {count} 样本 ({ratio:.2%})")
    return class_counts

# 3. 计算类别权重
def calculate_class_weights(class_counts, device):
    weights = 1.0 / class_counts.float()
    weights = weights / weights.sum() * len(weights)
    print(f"类别权重: {weights.tolist()}")
    return weights.to(device)

def create_weighted_sampler_safe(dataset, class_counts):
    """
    每个样本一个权重（适用于图分类，每个data.y是单个标签）
    """
    # 确保每个 data.y 是标量
    all_labels = torch.tensor([data.y.item() if data.y.numel() == 1 else int(data.y[0]) for data in dataset])
    sample_weights = 1.0 / class_counts[all_labels].float()
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(dataset),
        replacement=True
    )
    return sampler

# 5. 主程序
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")

    # 确保标签是 long 类型
    for data in dataTrain + dataTest:
        if data.y.dtype != torch.long:
            data.y = data.y.long()

    # 包装数据集
    train_dataset = MyGraphDataset(dataTrain)
    test_dataset = MyGraphDataset(dataTest)

    # 类别分布与权重
    train_class_counts = analyze_class_distribution(train_dataset.data_list, "训练集")
    test_class_counts = analyze_class_distribution(test_dataset.data_list, "测试集")
    class_weights = calculate_class_weights(train_class_counts, device)

    # 加权采样器
    sampler = create_weighted_sampler_safe(train_dataset, train_class_counts)

    # DataLoader
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=1,
        sampler=sampler,
        shuffle=False
    )
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=1,
        shuffle=False
    )

    print(f"训练集样本数: {len(train_dataset)}, 测试集样本数: {len(test_dataset)}")
    print("DataLoader 初始化完成")

    # 初始化模型
    model = GCNWithEdge(
        inFeat=dataTrain[0].x.shape[1],
        hiddenFeat=32,
        outFeat=16,
        embeddingDims=[20, 20],
        fchidden=32,
        vdim=2,
        finalDim=2,
        dropout=0.5
    ).to(device)

    # 损失函数和优化器
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=5, verbose=True
    )

    # 训练循环
    best_test_accuracy = 0
    best_metrics = None
    print("开始训练...")

    for epoch in range(50):
        train_loss, train_metrics = train_model(model, train_loader, optimizer, criterion, device)
        test_loss, test_metrics = test_model(model, test_loader, criterion, device)
        scheduler.step(test_loss)

        if test_metrics['accuracy'] > best_test_accuracy:
            best_test_accuracy = test_metrics['accuracy']
            best_metrics = test_metrics.copy()

        print(f"\nEpoch {epoch+1}:")
        print(f"学习率: {optimizer.param_groups[0]['lr']:.6f}")
        print(f"Train - Loss: {train_loss:.4f}, Acc: {train_metrics['accuracy']:.4f}")
        print(f"Test  - Loss: {test_loss:.4f}, Acc: {test_metrics['accuracy']:.4f}")
        print(f"Test  - Precision: {test_metrics.get('precision', 0):.4f}, Recall: {test_metrics.get('recall', 0):.4f}")

        if (epoch + 1) % 5 == 0:
            print_detailed_metrics(test_metrics, "Test")

    # 最终评估
    print("\n=== 最终评估结果 ===")
    print_detailed_metrics(test_metrics, "Final Test")
    print("\n=== 最佳模型结果 ===")
    print(f"最佳测试准确率: {best_test_accuracy:.4f}")
    if best_metrics:
        print_detailed_metrics(best_metrics, "Best Model")

    # 调整阈值
    def find_optimal_threshold(model, test_loader, device, target_precision=0.7):
        model.eval()
        all_probs = []
        all_targets = []
        with torch.no_grad():
            for data in test_loader:
                data = data.to(device)
                output = model(data)
                probs = torch.softmax(output, dim=1)
                all_probs.append(probs[:, 1])
                all_targets.append(data.y)
        all_probs = torch.cat(all_probs)
        all_targets = torch.cat(all_targets)

        thresholds = np.linspace(0.3, 0.9, 20)
        results = []
        for threshold in thresholds:
            preds = (all_probs > threshold).long()
            precision = precision_score(all_targets.cpu(), preds.cpu(), zero_division=0)
            recall = recall_score(all_targets.cpu(), preds.cpu(), zero_division=0)
            results.append((threshold, precision, recall))
            if precision >= target_precision:
                print(f"阈值 {threshold:.3f}: Precision={precision:.3f}, Recall={recall:.3f}")
        return results

    threshold_results = find_optimal_threshold(model, test_loader, device, target_precision=0.7)


使用设备: cuda

训练集 类别分布:
类别 0: 3659316 样本 (99.98%)
类别 1: 800 样本 (0.02%)

测试集 类别分布:
类别 0: 805613 样本 (99.97%)
类别 1: 256 样本 (0.03%)
类别权重: [0.00043714462663047016, 1.9995629787445068]
训练集样本数: 18, 测试集样本数: 6
DataLoader 初始化完成
开始训练...

Epoch 1:
学习率: 0.001000
Train - Loss: 15260.4419, Acc: 0.7526
Test  - Loss: 9654.7983, Acc: 0.5967
Test  - Precision: 0.0003, Recall: 0.4219

Epoch 2:
学习率: 0.001000
Train - Loss: 3615.7034, Acc: 0.8374
Test  - Loss: 15.8399, Acc: 0.9769
Test  - Precision: 0.0011, Recall: 0.0820

Epoch 3:
学习率: 0.001000
Train - Loss: 56.5402, Acc: 0.9796
Test  - Loss: 62.0999, Acc: 0.9800
Test  - Precision: 0.0013, Recall: 0.0820

Epoch 4:
学习率: 0.001000
Train - Loss: 42.0392, Acc: 0.9820
Test  - Loss: 62.1792, Acc: 0.9808
Test  - Precision: 0.0012, Recall: 0.0742

Epoch 5:
学习率: 0.001000
Train - Loss: 62.9231, Acc: 0.1274
Test  - Loss: 36.3368, Acc: 0.1144
Test  - Precision: 0.0003, Recall: 0.9297

Test 详细指标:
损失: 36.3368
准确率: 0.1144
精确率: 0.0003
召回率: 0.9297
F1分数: 0.0007

混淆矩阵:
真阳性(TP): 

In [28]:
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, average_precision_score, precision_score, recall_score, confusion_matrix

# ===== Focal Loss =====
class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2, alpha=None):
        super().__init__()
        self.gamma = gamma
        self.alpha = alpha  # Tensor of shape [num_classes]

    def forward(self, logits, targets):
        ce_loss = F.cross_entropy(logits, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return loss

# ===== 计算类别权重 =====
def calculate_class_weights(data_list, device):
    all_labels = torch.tensor([data.y.item() if data.y.numel() == 1 else int(data.y[0]) for data in data_list])
    class_counts = torch.bincount(all_labels)
    weights = 1.0 / class_counts.float()
    weights = weights / weights.sum() * len(weights)
    print(f"类别权重: {weights.tolist()}")
    return weights.to(device), class_counts

# ===== 安全加权采样器 =====
def create_weighted_sampler_safe(data_list, class_counts):
    all_labels = torch.tensor([data.y.item() if data.y.numel() == 1 else int(data.y[0]) for data in data_list])
    sample_weights = 1.0 / class_counts[all_labels].float()
    sampler = torch.utils.data.WeightedRandomSampler(
        weights=sample_weights,
        num_samples=len(data_list),
        replacement=True
    )
    return sampler

# ===== 找最优阈值 =====
def find_optimal_threshold(model, loader, device, target_precision=0.7):
    model.eval()
    all_probs, all_targets = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            probs = torch.softmax(output, dim=1)[:,1]  # 正类概率
            all_probs.append(probs.cpu())
            all_targets.append(data.y.cpu())
    all_probs = torch.cat(all_probs).numpy()
    all_targets = torch.cat(all_targets).numpy()

    thresholds = np.linspace(0.01, 0.5, 50)
    best_thresh = 0.5
    for th in thresholds:
        preds = (all_probs > th).astype(int)
        precision = precision_score(all_targets, preds, zero_division=0)
        if precision >= target_precision:
            best_thresh = th
            break
    return best_thresh

# ===== 主训练脚本 =====
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")

    # 确保标签是long类型
    for data in dataTrain + dataTest:
        if data.y.dtype != torch.long:
            data.y = data.y.long()

    # 计算类别权重
    class_weights, class_counts = calculate_class_weights(dataTrain, device)
    sampler = create_weighted_sampler_safe(dataTrain, class_counts)

    # 创建 DataLoader
    train_loader = DataLoader(dataTrain, batch_size=1, sampler=sampler)
    test_loader = DataLoader(dataTest, batch_size=1, shuffle=False)

    # 初始化模型
    model = GCNWithEdge(
        inFeat=dataTrain[0].x.shape[1],
        hiddenFeat=32,
        outFeat=16,
        embeddingDims=[20,20],
        fchidden=32,
        vdim=2,
        finalDim=2,
        dropout=0.5
    ).to(device)

    # 损失函数和优化器
    criterion = FocalLoss(gamma=2, alpha=class_weights)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)

    # 训练循环
    best_f1 = 0
    best_metrics = None
    for epoch in range(50):
        # ===== 训练 =====
        model.train()
        total_loss = 0
        for data in train_loader:
            data = data.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, data.y)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        train_loss = total_loss / len(train_loader)

        # ===== 测试 =====
        model.eval()
        all_probs, all_targets = [], []
        with torch.no_grad():
            for data in test_loader:
                data = data.to(device)
                output = model(data)
                probs = torch.softmax(output, dim=1)[:,1]
                all_probs.append(probs.cpu())
                all_targets.append(data.y.cpu())
        all_probs = torch.cat(all_probs).numpy()
        all_targets = torch.cat(all_targets).numpy()
        all_preds = (all_probs > 0.5).astype(int)

        test_f1 = f1_score(all_targets, all_preds, zero_division=0)
        test_auprc = average_precision_score(all_targets, all_probs)
        test_accuracy = (all_preds == all_targets).mean()

        # 保存最佳模型（F1为主）
        if test_f1 > best_f1:
            best_f1 = test_f1
            best_metrics = {
                'f1': test_f1,
                'auprc': test_auprc,
                'accuracy': test_accuracy
            }
            torch.save(model.state_dict(), 'best_model_f1.pth')

        print(f"Epoch {epoch+1}: TrainLoss={train_loss:.4f}, TestF1={test_f1:.4f}, AUPRC={test_auprc:.4f}, Acc={test_accuracy:.4f}")

    # ===== 最终输出 =====
    print("\n=== 最佳模型指标 ===")
    print(best_metrics)

    # ===== 自动寻找最佳阈值 =====
    best_threshold = find_optimal_threshold(model, test_loader, device, target_precision=0.7)
    print(f"最佳阈值(精确率≥0.7): {best_threshold:.3f}")

使用设备: cuda
类别权重: [1.0, 1.0]


d:\anaconda\envs\pytorchenv\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1: TrainLoss=0.6990, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 2: TrainLoss=0.4615, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 3: TrainLoss=0.2100, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 4: TrainLoss=0.1425, TestF1=0.0000, AUPRC=0.0004, Acc=0.9997
Epoch 5: TrainLoss=0.1446, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 6: TrainLoss=0.1467, TestF1=0.0000, AUPRC=0.0004, Acc=0.9997
Epoch 7: TrainLoss=0.1425, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 8: TrainLoss=0.1241, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 9: TrainLoss=0.0931, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 10: TrainLoss=0.0975, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 11: TrainLoss=0.1260, TestF1=0.0000, AUPRC=0.0004, Acc=0.9997
Epoch 12: TrainLoss=0.1280, TestF1=0.0000, AUPRC=0.0004, Acc=0.9997
Epoch 13: TrainLoss=0.0943, TestF1=0.0000, AUPRC=0.0003, Acc=0.9997
Epoch 14: TrainLoss=0.0856, TestF1=0.0000, AUPRC=0.0004, Acc=0.9997
Epoch 15: TrainLoss=0.0898, TestF1=0.0000, AUPRC=0.0004, 

In [11]:
# 建立baseline模型
# 将卷积层和边信息结合，在这里我们先不考虑时间顺序，只是搭建一个baseline模型

from torch.nn import Module
from torch_geometric.nn import GCNConv
from torch import nn

'''
infeat:GCN层输入维数
hiddenFeat:GCN层隐含维数
outFeat:GCN层输出维数
embeddingDims:属性数据嵌入维数
fchidden:属性数据输出隐藏层维数
vdim:数值数据维数
finalDim:最后的输出层维数
'''
class GCNWithEdge(nn.Module):
    def __init__(self,
                 inFeat,
                 hiddenFeat,
                 outFeat,
                 embeddingDims = [20,20],
                 fchidden = 32,
                 vdim = 2,
                 finalDim = 2,
                 dropout = 0.5):
        super(GCNWithEdge, self).__init__()
        
        # 由于数据集较大，故我们对于GCN要求的层数不能过多，在这里做为baseline我们只要求有两层gcn链接
        self.conv1 = GCNConv(inFeat, hiddenFeat)
        self.conv2 = GCNConv(hiddenFeat, outFeat)

        # 构建处理属性数据层
        self.embeddingList = nn.ModuleList()
        for dim in embeddingDims:
            self.embeddingList.append(torch.nn.Embedding(dim, fchidden))
        
        self.finalLayer = nn.Sequential(
            nn.Linear(outFeat + len(embeddingDims)*fchidden + vdim, finalDim),
            nn.ReLU(),
            nn.Dropout(dropout),    
            nn.Linear(finalDim, 2)  # 假设二分类问题
        )
    
    def forward(self, 
                data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr

        # gcn得到节点特征
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index) # x : [num_node,outFeat]

        # 将节点数据转换为边数据
        edgeIndex = data.edge_index
        start,end = edgeIndex
        node2Edge = x[start] - x[end]  #node2Edge : [num_edge,outFeat]

        # 提取属性数据
        edgeAttr = edge_attr.T # edgeAttr : [num_edge, num_attr]
        vlData = edgeAttr[:, :2].float()

        for i in range(len(self.embeddingList)):
            emb = self.embeddingList[i]
            vlData = torch.cat([vlData, emb(edgeAttr[:, i+2].long())], dim=1)
        
        total = torch.cat([node2Edge, vlData], dim=1)

        out = self.finalLayer(total)

        return F.log_softmax(out, dim=1)


In [13]:
model = GCNWithEdge(
    inFeat=dataTrain[0].x.shape[1],
    hiddenFeat=32,
    outFeat=16,
    embeddingDims=[20, 20],
    fchidden=32,
    vdim=2,
    finalDim=2,
    dropout=0.5
)

In [14]:
out = model(dataTrain[0])

In [15]:
print(out)

tensor([[-5.3382e-01, -8.8276e-01],
        [-1.2078e+02,  0.0000e+00],
        [-2.9309e+01,  0.0000e+00],
        ...,
        [-2.3299e+03,  0.0000e+00],
        [-1.0671e+00, -4.2162e-01],
        [-5.3382e-01, -8.8276e-01]], grad_fn=<LogSoftmaxBackward0>)
